# Stream power, channel steepness, and relief HW

### From the original notebook
The equation at hand
$$
\begin{equation}
 \frac{d z}{d t} = -K_\text{sp} A^{m_{sp}} S^{n_{sp}} + U
\end{equation}
$$
Here, $K_{sp}$ is the erodibility coefficient on fluvial incision, which is thought to be positively correlated with climate wetness, or storminess (this is hard to quantify) and to be negatively correlated with rock strength (again, rock strength is hard to quantify). $m_{sp}$ and $n_{sp}$ are positive exponents, usually thought to have a ratio, $m_{sp}/n_{sp} \approx 0.5$. $A$ is drainage area and $S$ is the slope of steepest descent ($-\frac{dz}{dx}$) where $x$ is horizontal distance (positive in the downslope direction) and $z$ is elevation. (If slope is negative there is no fluvial erosion.) $U$ is an externally-applied rock uplift rate field.

The fluvial erosion term is also known as the stream power equation. Before using this notebook you should be familiar with this equation from class lectures and reading. 

**STUDENTS: Questions to answer before starting this assignment.**

Answer these questions before running the notebook.

1. What do you think will happen to total relief (defined as the maximum minus the minimum elevation, here area is fixed) and channel slope at steady state if $K_{sp}$ is uniformly increased?

*Since at steady state*
$$
U = K_\text{sp} A^{m_{sp}} S^{n_{sp}}
$$
*where $U, A, m_{sp}, n_{sp}$ are constant and the only variables allowed to change are $K_\text{sp}$ and $S$, then to keep the equation balanced if we increase $K_\text{sp}$ the value of $S$ should decrease, and therefore we will get less relief.* 


2. What do you think will happen to total relief and channel slope at steady state if $U$ is uniformly increased?

*Using a similar logic if $K_\text{sp}$ is now constant and we increase $U$ then the value of $S$ should increase to mantain balance, meaning that the relief is likely to go up.*

### I addapted the code to make it more comfortable to me

In [ ]:
# imports
import numpy as np
from matplotlib import pyplot as plt

from landlab import RasterModelGrid, imshow_grid
from landlab.components import (
    ChannelProfiler,
    ChiFinder,
    FlowAccumulator,
    SteepnessFinder,
    StreamPowerEroder,
)
from landlab.io import write_esri_ascii

In [ ]:
# global parameters
params = { "timer":0 }
raster = 0

In [ ]:
# set parameters function
def set_params( tmax=1e6, uplift_rate = 0.0001, **kwargs):
    """set some parameters. All parameters can be set by passing the **kwargs if needed"""
    params["rows"] = 100
    params["columns"] = 100
    params["dxy"] = 100
    params["timer"] = 0
    params["tmax"] = tmax
    params["uplift_rate"] = uplift_rate
    params["dt"] = 5000
    params["tmax"] = tmax
    params["K_sp"] = 1.0e-5
    params["m_sp"] = 0.5
    params["n_sp"] = 1.0
    if kwargs:
        print("additional parameters passed")
        for key in kwargs:
            params[key]=kwargs[key]
    params["times"] = len(np.arange(0, tmax, params["dt"]))
# run at least once
set_params()

In [ ]:
# reset grid, flow route tools, eroder and all landlab tools
def reset_grid():
    """
    reset grid, flow route tools, eroder and all landlab tools.
    
    Returns the raster and the associated topography, flow_router, stream_eroder,steep_finder,chi_finder 
    """
    raster = RasterModelGrid((params["rows"], params["columns"]), params["dxy"])
    raster.set_closed_boundaries_at_grid_edges(True, True, True, False)
    np.random.seed(77)
    noise = np.random.rand(raster.number_of_nodes) / 1000.0
    topography = raster.add_zeros("topographic__elevation", at="node")
    topography += noise
    flow_router = FlowAccumulator(raster, flow_director='FlowDirectorD8')
    stream_eroder = StreamPowerEroder(raster, K_sp=params["K_sp"], 
                            m_sp=params["m_sp"], n_sp=params["n_sp"],
                            threshold_sp=0.0)
    params["theta"] = params["m_sp"] / params["n_sp"]
    steep_finder = SteepnessFinder(raster, reference_concavity=params["theta"], min_drainage_area=1000.0)
    chi_finder = ChiFinder(raster,
                min_drainage_area=1000.0,
                reference_concavity=params["theta"],
                use_true_dx=True)
    return (raster, topography, flow_router, stream_eroder, steep_finder, chi_finder)
# run at least once
raster, z, frr, spr, sf, cf = reset_grid()

def channel_profiler():
    return ChannelProfiler(raster,
                      number_of_watersheds=3,
                      main_channel_only=True,
                      minimum_channel_threshold=params["dxy"]**2)

In [ ]:
# Evolve the system
def evolve(time = 0, omit_times = True):
    if ("times" in params.keys()) and (not time):
        time = params["times"]
    for ti in range(time):
        uplift = np.ones(raster.number_of_nodes) * params["uplift_rate"]
        z[raster.core_nodes] += uplift[raster.core_nodes] * params["dt"]
        frr.run_one_step()
        spr.run_one_step(params["dt"])
        params["timer"] += params["dt"]
        if not omit_times:
            print(params["timer"])
    print(params["timer"])

In [ ]:
# plotting functions
def plot_topography():
    plt.figure()
    imshow_grid(raster,
                "topographic__elevation",
                grid_units=("m", "m"),
                var_name="Elevation (m)")
    title_text = f'K_{{sp}}={params["K_sp"]}; U={params["uplift_rate"]*1000} mm/yr; time={params["timer"]/1000} kyr'
    plt.title(title_text)
    max_elev = np.max(z)
    print("Maximum elevation is ", np.max(z))

def plot_slopes_area():
    plt.figure()
    plt.loglog(
        raster.at_node["drainage_area"][np.where(raster.node_y[raster.core_nodes]>200)],
        raster.at_node["topographic__steepest_slope"][np.where(raster.node_y[raster.core_nodes]>200)],
        "b.",
    )
    plt.ylabel("Topographic slope")
    plt.xlabel("Drainage area (m^2)")
    title_text = f'$K_{{sp}}$={params["K_sp"]}; $U$={params["uplift_rate"]*1000} mm/yr; $time$={params["timer"]/1000} kyr'

    plt.title(title_text)

def plot_profiles(channel_profiler):
    plt.figure()
    title_text = f'$K_{{sp}}$={params["K_sp"]}; $U$={params["uplift_rate"]*1000} mm/yr; $time$={params["timer"]/1000} kyr'
    channel_profiler.plot_profiles(xlabel='distance upstream (m)',
                    ylabel='elevation (m)',
                    title=title_text)
    ax=plt.gca()
    ax.invert_xaxis()

    plt.figure()
    channel_profiler.plot_profiles_in_map_view()

    # slope-area data in just the profiled channels
    plt.figure()
    for i, outlet_id in enumerate(channel_profiler.data_structure):
        for j, segment_id in enumerate(channel_profiler.data_structure[outlet_id]):
            if j == 0:
                label = "channel {i}".format(i=i + 1)
            else:
                label = '_nolegend_'
            segment = channel_profiler.data_structure[outlet_id][segment_id]
            profile_ids = segment["ids"]
            color = segment["color"]
            plt.loglog(
                raster.at_node["drainage_area"][profile_ids],
                raster.at_node["topographic__steepest_slope"][profile_ids],
                '.',
                color=color,
                label=label,
            )

    plt.legend(loc="lower left")
    plt.xlabel("drainage area (m^2)")
    plt.ylabel("channel slope [m/m]")
    title_text = f'$K_{{sp}}$={params["K_sp"]}; $U$={params["uplift_rate"]*1000} mm/yr; $time$={params["timer"]/1000} kyr'
    plt.title(title_text)

def plot_chi(channel_profiler):
    cf.calculate_chi()

    # # chi-elevation plots in the profiled channels
    plt.figure()

    for i, outlet_id in enumerate(channel_profiler.data_structure):
        for j, segment_id in enumerate(channel_profiler.data_structure[outlet_id]):
            if j == 0:
                label = "channel {i}".format(i=i + 1)
            else:
                label = '_nolegend_'
            segment = channel_profiler.data_structure[outlet_id][segment_id]
            profile_ids = segment["ids"]
            color = segment["color"]
            plt.plot(
                raster.at_node["channel__chi_index"][profile_ids],
                raster.at_node["topographic__elevation"][profile_ids],
                color=color,
                label=label,
            )

    ax=plt.gca()
    ax.invert_xaxis()
    plt.xlabel("chi index (m)")
    plt.ylabel("elevation (m)")
    plt.legend(loc="upper right")
    title_text = f'$K_{{sp}}$={params["K_sp"]}; $U$={params["uplift_rate"]} m/yr; $time$={params["timer"]/1000} kyr; concavity={params["m_sp"]/params["n_sp"]}'
    plt.title(title_text)

    # chi map
    plt.figure()
    imshow_grid(
        raster,
        "channel__chi_index",
        grid_units=("m", "m"),
        var_name="Chi index (m)",
        cmap="jet",
    )
    title_text = f'$K_{{sp}}$={params["K_sp"]}; $U$={params["uplift_rate"]} m/yr; $time$={params["timer"]/1000} kyr; concavity={params["m_sp"]/params["n_sp"]}'
    plt.title(title_text)

def plot_steepness(channel_profiler):
    sf.calculate_steepnesses()

    plt.figure()

    for i, outlet_id in enumerate(channel_profiler.data_structure):
        for j, segment_id in enumerate(channel_profiler.data_structure[outlet_id]):
            if j == 0:
                label = "channel {i}".format(i=i + 1)
            else:
                label = '_nolegend_'
            segment = channel_profiler.data_structure[outlet_id][segment_id]
            profile_ids = segment["ids"]
            distance_upstream = segment["distances"]
            color = segment["color"]
            plt.plot(
                distance_upstream,
                raster.at_node["channel__steepness_index"][profile_ids],
                'x',
                color=color,
                label=label,
            )

    ax=plt.gca()
    ax.invert_xaxis()
    plt.xlabel("distance upstream (m)")
    plt.ylabel("steepness index")
    plt.legend(loc="upper right")
    plt.title(f'$K_{{sp}}$={params["K_sp"]}; $U$={params["uplift_rate"]*1000} mm/yr; $time$={params["timer"]/1000} kyr; concavity={params["m_sp"]/params["n_sp"]}')

    # channel steepness map
    plt.figure()
    imshow_grid(
        raster,
        "channel__steepness_index",
        grid_units=("m", "m"),
        var_name="Steepness index ",
        cmap="jet",
    )
    plt.title(f'$K_{{sp}}$={params["K_sp"]}; $U$={params["uplift_rate"]*1000} mm/yr; $time$={params["timer"]/1000} kyr; concavity={params["m_sp"]/params["n_sp"]}')

In [ ]:
evolve()
plot_topography()

In [ ]:
prf = channel_profiler()
prf.run_one_step()
plot_profiles(prf)

In [ ]:
plot_chi(prf)

In [ ]:
plot_steepness(prf)

#### 1. **Steady state with low rock uplift rate.** 
Using the parameters provided in the initial notebook, run the landscape to steady state. How did you decide that the landscape reached steady state? [These landscapes may not reach a perfect steady state. Close is fine.] Include appropriate plots. 


#### 2. **Steady-state landscape with higher rock uplift rate.**

Now increase rock uplift uniformly by a factor of 2 to 0.0002 m/yr. Rerun the entire notebook, and continue to run the main evolution loop until the landscape reaches steady state. Provide a plot that illustrates that the landscape is in steady state. What aspects of the landscape have changed in comparison with the base landscape from question 1?

#### 3. **Increase erodibility.** 

Now increase $K_{sp}$ to 2E-5. Make sure rock uplift rate is set to the original value of 0.0001 m/yr. Rerun the entire notebook, and continue to run the main evolution loop until the landscape reaches steady state. Compare this landscape with the one produced in the first question. What, if anything, has changed? 

#### 4. **General Reflection**

In 4 sentences or less, describe how the steady-state landscape form changes with rock uplift rate? What metrics could you measure in the landscape that would illustrate higher rock uplift rate? In 4 sentences or less, describe how the steady-state landscape form changes with erodibility? What metrics could you measure in different landscapes that would illustrate they have different erodibilities? What might lead to different erodibilities in different landscapes?

#### 5. **Final Reflection**

Was your initial insight into how parameters would affect the landscape correct? Discuss in 6 sentences or less.